In [1]:
%cd ..

/kaggle/working


In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../cand_unsupervised/transition_prob"):
    cfg = compose(config_name="config.yaml")
    print(OmegaConf.to_yaml(cfg))

debug: false
seed: 7
dir:
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
  exp_dir: /kaggle/working/output/exp
  cand_unsupervised_dir: /kaggle/working/output/cand_unsupervised
  cand_supervised_dir: /kaggle/working/output/cand_supervised
exp:
  range_transition: 1
  num_candidate: 100
  k:
  - 1
  - 5
  - 10
  - 50
  - 100



In [11]:
from pathlib import Path

import numpy as np
import polars as pl

import utils
from utils.load import load_label_data, load_log_data, load_yad_data
from utils.metrics import calculate_metrics

In [6]:
with utils.timer("load data"):
    train_label_df = load_label_data(Path(cfg.dir.data_dir))

[load data] done in 0.0 s


In [13]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = train_label_df.get_column("session_id").to_numpy()
y = train_label_df.get_column("yad_no").to_numpy()
folds = np.zeros(train_label_df.height)
for fold, (train_index, valid_index) in enumerate(skf.split(X, y)):
    folds[valid_index] = fold

train_fold_df = train_label_df.with_columns(
    pl.Series(folds).cast(pl.Int64).alias("fold")
).select(["session_id", "fold"])

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [14]:
train_fold_df

session_id,fold
str,i64
"""000007603d533d…",1
"""0000ca043ed437…",1
"""0000d4835cf113…",4
"""0000fcda1ae1b2…",1
"""000104bdffaaad…",0
"""00011afe25c343…",1
"""000125c737df18…",3
"""0001763050a10b…",3
"""000178c4d4d567…",2
